In [1]:
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
import geocoder

In [2]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
location = get_latlng('M4G')

In [3]:
# To save file
TorontoData  = pd.read_csv('Geographical coordinates of postal code.csv')
TorontoData.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.696665,-79.260163


In [4]:
# To see random portion of table
refined_table = TorontoData.sample(frac=1)
refined_table.reset_index(inplace=True)
refined_table.drop('index', axis=1, inplace=True)

print("This is the required table.")
print()
refined_table.head(12)

This is the required table.



,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront , Regent Park",43.655120,-79.362640
1,M1W,Scarborough,L'Amoreaux West,43.800881,-79.320740
2,M6R,West Toronto,"Parkdale , Roncesvalles",43.647870,-79.449762
3,M4X,Downtown Toronto,"Cabbagetown , St. James Town",43.668160,-79.366602
4,M3N,North York,Downsview Northwest,43.755331,-79.519590
5,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481
6,M4C,East York,Woodbine Heights,43.689665,-79.307169
7,M6A,North York,"Lawrence Heights , Lawrence Manor",43.723125,-79.451589
8,M6B,North York,Glencairn,43.707990,-79.448367
9,M5L,Downtown Toronto,"Commerce Court , Victoria Hotel",43.648205,-79.378793


### Find number of neighbors

In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorontoData ['Borough'].unique()),
        TorontoData .shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [6]:
toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = TorontoData['Latitude']
Y = TorontoData['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
TorontoData['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(TorontoData['Latitude'], TorontoData['Longitude'], TorontoData['Borough'], TorontoData['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map